In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import xgboost as xgb

import wandb
from wandb.lightgbm import wandb_callback

pd.set_option("display.max_columns", 181)
pd.set_option("display.min_rows", 200)

In [34]:
data_dictionary = pd.read_csv("DataDictionaryWiDS2021.csv")
unlabeled = pd.read_csv("UnlabeledWiDS2021.csv")
training = pd.read_csv("TrainingWiDS2021.csv")

del training['Unnamed: 0']
del unlabeled['Unnamed: 0']

column_datatype_mapping = dict(zip(data_dictionary['Variable Name'], data_dictionary['Data Type']))

all_data = training.append(unlabeled).drop(['encounter_id', 
                                            'hospital_id', 
                                            'diabetes_mellitus'], axis=1)

In [7]:
def age_group(x):
    
    if x<=50:
        return "0-50"
    elif x>50 and x<=65:
        return "50-65"
    elif x>65 and x<=75:
        return "65-75"
    elif x>75:
        return "75+"

    
def get_bmi_cat(x):
    
    if x<= 18.5:
        return "underweight"
    elif x> 18.5 and x<= 25:
        return "normal"
    elif x> 25 and x<= 30:
        return "overweight"
    elif x> 30 and x<= 35:
        return "obese_class1"
    elif x> 35 and x<= 40:
        return "obese_class2"
    elif x> 40:
        return "obese_class3"

def _group_feature_eng(combined_df, n_train, group_var, num_feats):
    
    """
    combined_df: the combined train & test datasets.
    n_train: number of training observations
    group_var: the variable we'd like to group by
    num_feat: numerical features
    
    This function loops through all numerical features, 
    group by the variable and compute new statistics of the numerical features.
    """
    
    grouped = combined_df.groupby(group_var)

    for nf in num_feats:

        combined_df[group_var + '_' + nf + '_max'] = grouped[nf].transform('max')
        combined_df[group_var + '_' + nf + '_min'] = grouped[nf].transform('min')
        combined_df[group_var + '_' + nf + '_mean'] = grouped[nf].transform('mean')
        combined_df[group_var + '_' + nf + '_skew'] = grouped[nf].transform('skew')
        combined_df[group_var + '_' + nf + '_std'] = grouped[nf].transform('std')

    train_X = combined_df.iloc[:n_train]
    test_X = combined_df.iloc[n_train:]
    
    return train_X, test_X

In [35]:
all_data.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,NaN,NaN,159.0,67.0,159.0,67.0,36.3,35.1

In [36]:
'''num_feats = set([c for c in all_data.columns if 'glucose' in c])
n_train = training.shape[0]
n_test = unlabeled.shape[0]

# create new categorical variables
all_data['age_group'] = all_data['age'].apply(age_group)
all_data['bmi_group'] = all_data['bmi'].apply(get_bmi_cat)
'''
cat_cols = ['ethnicity', 'gender', 'hospital_admit_source',
           'icu_admit_source', 'icu_stay_type', 'icu_type',
           'apache_2_diagnosis', 'apache_3j_diagnosis','icu_id']
           #'age_group','bmi_group']

binary_cols = data_dictionary.loc[data_dictionary['Data Type'] == 'binary','Variable Name'].values
num_cols = data_dictionary.loc[data_dictionary['Data Type'] == 'numeric','Variable Name'].values

    

for col in all_data.columns:
    if col in cat_cols:
        print(col)
        all_data[col] = LabelEncoder().fit_transform(all_data[col].astype('str'))
        all_data[col]= all_data[col].astype('category')        

ethnicity
gender
hospital_admit_source
icu_admit_source
icu_id
icu_stay_type
icu_type
apache_2_diagnosis
apache_3j_diagnosis


In [12]:
# create new features from newly created categorical variables
age_group_feat_eng = True
bmi_group_feat_eng = True

if age_group_feat_eng:
    df_train, df_pred = _group_feature_eng(all_data, n_train, 'age_group', num_feats)

if bmi_group_feat_eng:
    df_train, df_pred = _group_feature_eng(all_data, n_train, 'bmi_group', num_feats)

In [39]:
df_train = all_data[:len(training)]
df_pred = all_data[len(training):].reset_index(drop=True)
Y = training['diabetes_mellitus']

In [40]:
X_train, X_val, y_train, y_val = train_test_split(df_train, Y, test_size=0.30, random_state=42,shuffle=True )

In [43]:
xg_train = xgb.DMatrix(df_train.values, label=Y)
xg_test = xgb.DMatrix(X_val.values, label=y_val)

wandb.init(project="wids_2021", sync_tensorboard=True)
config = wandb.config


param = {'subsample': 1.0, 
         'num_leaves': 10, 
         'min_child_weight': 1, 
         'max_depth': 6, 
         'learning_rate': 0.1, 
         'gamma': 1.5, 
         'colsample_bytree': .6,
         'n_estimators':600,
          'eval_metric':'auc',
          'objective': 'binary:logistic'}

# wandb.config.update(params)

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 600
bst = xgb.train(param, xg_train, num_round, watchlist, callbacks=[wandb.xgboost.wandb_callback()])

# get prediction
pred = bst.predict(xg_test)

train-auc,0.93674
test-auc,0.93625
_step,599
_runtime,233
_timestamp,1611776090


train-auc,▁▁▂▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████
test-auc,▁▁▂▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: wandb version 0.10.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[19:37:03] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.79998	test-auc:0.79649
[1]	train-auc:0.80851	test-auc:0.80572
[2]	train-auc:0.82457	test-auc:0.82179
[3]	train-auc:0.83108	test-auc:0.82817
[4]	train-auc:0.83312	test-auc:0.83047
[5]	train-auc:0.83467	test-auc:0.83181
[6]	train-auc:0.83629	test-auc:0.83355
[7]	train-auc:0.83851	test-auc:0.83578
[8]	train-auc:0.83981	test-auc:0.83721
[9]	train-auc:0.84070	test-auc:0.83841
[10]	train-auc:0.84180	test-auc:0.83968
[11]	train-auc:0.84236	test-auc:0.84017
[12]	train-auc:0.84320	test-auc:0.84091
[13]	train-auc:0.84528	test-auc:0.84281
[14]	train-auc:0.84645	test-auc:0.84419
[15]	train-auc:0.84705	test-auc:0.84464
[16]	train-auc:0.84794	tes

In [44]:
pred_proba = bst.predict(xgb.DMatrix(df_pred.values))

submittion = pd.DataFrame([unlabeled.encounter_id,pred_proba]).T
submittion.encounter_id = submittion.encounter_id.astype('int32')
submittion.set_index('encounter_id',inplace=True)
submittion.columns = ['diabetes_mellitus']
submittion.to_csv('submissions/SolutionWiDS2021_XGB_600_epochs_nans.csv')